In [ ]:
import numpy as np
import xarray as xr
from bokeh.embed import json_item
import pandas as pd
# imports for plotting tool
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Column
from bokeh.models.tools import HoverTool
# from bokeh.layouts import widgetbox
from bokeh.models.widgets import Div
from bokeh.models.widgets import Paragraph
from bokeh.models.widgets import PreText
# import to generate html tables
from json2html import *
# imports needed when debugging into a notebook
from IPython.core.display import HTML, display_html

In [ ]:
def get_plottable_variables(nc_url):
    DS = xr.open_dataset(nc_url)
    return {"y_axis":[i for i in DS if len(DS[i].shape) == 1]}

def get_data(nc_url, nc_variable, resample=None):
    DS = xr.open_dataset(nc_url)
    df = DS.to_dataframe()
    df.replace(9.96921e+36, np.NaN, inplace=True)
    data = df[nc_variable]
    if resample:
        data = data.resample(resample).mean()
    data=pd.DataFrame(data)
    data.dataset_metadata = ''
    data.dataset_metadata = DS.attrs
    data.variable_metadata = ''
    data.variable_metadata = DS[nc_variable].attrs
    return data

def create_plot(data):
    data['tooltip'] = [x.strftime("%Y-%m-%d %H:%M:%S") for x in data.index]
    source = ColumnDataSource(data)
    tools_to_show = "box_zoom, pan,save, hover, reset, wheel_zoom"
    var_label = '@'+str(data.columns[0])
    p = figure(toolbar_location="above", 
               x_axis_type="datetime", 
               tools=tools_to_show) # 
    tooltips = [('Time', '@tooltip'), 
                (str(data.variable_metadata['long_name']), var_label)
               ]
    hover = p.select(dict(type=HoverTool))
    hover.formatters = {'tooltip': "datetime"} 
    hover.tooltips = tooltips
    p.line(x='time', 
           y=data.columns[0], #data.variable_metadata['standard_name'],
           source=source,
           color='green',
           legend_label=data.columns[0],
          )
    return p

In [ ]:
from bokeh.io import push_notebook, show, output_notebook
output_notebook()

In [ ]:
resource_url = 'http://hyrax:8080/opendap/SN99938.nc'


In [ ]:
variables = get_plottable_variables(resource_url)
variables['y_axis']

In [ ]:
data = get_data(resource_url, variables['y_axis'][2])
plot = create_plot(data)

In [ ]:
title = data.variable_metadata['long_name']
summary = data.dataset_metadata['summary']
dataset_metadata = json2html.convert(json = data.dataset_metadata)
variable_metadata = json2html.convert(json = data.variable_metadata)

In [ ]:
variables['y_axis'][2]


In [ ]:
data.columns[0]


In [ ]:
HTML(variable_metadata)


In [ ]:
title_div = Div(text="""<h1>
                            <b>
                                {title}
                            </b>
                        </h1>""".format(title=title))

abstract_text = Div(text="""<details>
                                <summary>
                                    Summary
                                </summary>
                                <p>
                                    {summary}
                                </p>
                            </details>""".format(summary=summary),
                    width=400, 
                    height=200)

dataset_metadata_div = Div(text="""<details>
                                       <summary>
                                           Dataset Metadata
                                       </summary>
                                       <p>
                                           {dataset_metadata}
                                       </p>
                                    </details>""".format(dataset_metadata=dataset_metadata), 
                           width=300, 
                           height=100)

variable_metadata_div = Div(text="""<details>
                                        <summary>
                                            Variable Metadata
                                        </summary>
                                        <p>
                                            {variable_metadata}
                                        </p>
                                    </details>""".format(variable_metadata=variable_metadata), 
                            width=200, 
                            height=100)

plot_caption = Paragraph(text="""This text can be a figure caption.""", 
                         width=200, 
                         height=100)

In [ ]:
show(Column(title_div, abstract_text, plot, plot_caption, variable_metadata_div, dataset_metadata_div))
